<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
# Import the neccesary python libraries for Bayesian analysis
# and data visualisation.
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import os
import pystan as ps
import arviz as az
import statistics as Stats
import scipy.stats as stats
from patsy import dmatrix

In [41]:
os.chdir(r"C:\Users\harri\OneDrive\Documents\Datasets practice")

In [42]:
df =  pd.read_csv("Sleepstudy_exported")
# Convert to long to change the particapnt ids to 1:18
df_wide = df.pivot_table(index = 'Subject', columns = 'Days', values = 'Reaction').reset_index()
df_wide['Subject'] = list(range(1, len(df_wide)+1))

In [59]:
df_long = pd.melt(df_wide, id_vars=['Subject'], value_vars = range(0,10),
        var_name='Day', value_name='Reaction')
df_long['Day'] = df_long['Day'].astype(int)

In [82]:
Hierachial_regression = """

data {
  int<lower=0> N; // number of data observed
  int<lower=0> J; // number of subjects
  int<lower= 0> K; // number of fixed effects
  vector[N] y; // obsevered data
  vector[N] x; // predictor
  int<lower = 0, upper=J> subj_id[N];
  
}

parameters {
vector[K] beta;
vector[J] u_1;
vector[J] u_2;
real<lower = 0> sigma;
vector<lower=0>[2] sigma_u;


}

transformed parameters{
vector[N] mu;

for (i in 1:N){
mu[i] = beta[1] + u_1[subj_id[i]] + (beta[2] + u_2[subj_id[i]]) * x[i];
    }
}

model {
// Priors
beta[1] ~ normal(250,50);
beta[2] ~ normal(0,50);
sigma_u[1] ~ normal(0, 50);
sigma_u[2] ~ normal(0, 20);
u_1 ~ normal(0, sigma_u[1]);
u_2 ~ normal(0, sigma_u[2]);


//likelihood
y ~ normal(mu, sigma);
}

"""

In [83]:
sm = ps.StanModel(model_code=Hierachial_regression)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_348aeac0022a9ac8edcc1e70cf12c1cc NOW.


In [77]:
X = df_long['Day']
data = {'N': len(df),
        'y': df_long.Reaction.values,
        'K': 2,
        'x': X,
        'J': df_long.Subject.nunique(),
        'subj_id': df_long.Subject.values}

In [84]:
fit = sm.sampling(data = data, iter = 2000, warmup=1000, seed = 1)

In [86]:
#print(fit)